# Transfer Learning for Image Classification

In [1]:
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten,Input
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16

E:\programfiles\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
num_classes=3
img_size=224
image_input = Input(shape=(224, 224, 3))
model = VGG16(input_tensor=image_input, include_top=False, weights= 'imagenet')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [5]:
top_model = Sequential()
top_model.add(model)
top_model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=model.output_shape[1:],padding='same'))
top_model.add(MaxPooling2D((2,2), padding='same'))
top_model.add(Dropout(0.25))
top_model.add(Conv2D(64, kernel_size= (3,3),activation='relu',padding='same'))
top_model.add(MaxPooling2D((2,2), padding='same'))
top_model.add(Flatten())
top_model.add(Dense(128,activation='relu'))
top_model.add(Dense(num_classes,activation='softmax'))

In [6]:
top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 32)          147488    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 64)          18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 256)               0         
__________

In [7]:
for layer in top_model.layers[:-8]:
    layer.trainable = False
    
    
top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 32)          147488    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 64)          18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 256)               0         
__________

In [8]:
top_model.compile(optimizer = 'Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
from keras.callbacks import EarlyStopping

earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=3,
                          verbose=1, mode='auto')
callbacks_list = [earlystop]

batch_size=32

data_generator = ImageDataGenerator(rescale=1./255)

data_generator_with_aug = ImageDataGenerator(horizontal_flip=True,
                                            rescale=1./255,
                                            width_shift_range=0.2,
                                            height_shift_range=0.2)

train_generator = data_generator_with_aug.flow_from_directory(r'E:\Thesis\Try1\Train',
                                                             batch_size=batch_size,
                                                             target_size=(img_size,img_size),
                                                             class_mode='categorical')

validation_generator = data_generator.flow_from_directory(r'E:\Thesis\Try1\Test',
                                                         batch_size=batch_size,
                                                         target_size=(img_size,img_size),
                                                         class_mode='categorical') 
history = top_model.fit_generator(train_generator,
                   steps_per_epoch=10,
                   epochs=50,
                   validation_data = validation_generator,
                   validation_steps = 1,
                   callbacks=callbacks_list)

Found 450 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Epoch 1/50
10/10 [==============================] - 98s 10s/step - loss: 1.1700 - acc: 0.3549 - val_loss: 1.0923 - val_acc: 0.3833
Epoch 2/50
10/10 [==============================] - 101s 10s/step - loss: 1.0450 - acc: 0.4688 - val_loss: 1.0110 - val_acc: 0.6167
Epoch 3/50
10/10 [==============================] - 93s 9s/step - loss: 0.9106 - acc: 0.5662 - val_loss: 0.8076 - val_acc: 0.7000
Epoch 4/50
10/10 [==============================] - 95s 9s/step - loss: 0.7721 - acc: 0.6056 - val_loss: 0.6728 - val_acc: 0.7500
Epoch 5/50
10/10 [==============================] - 94s 9s/step - loss: 0.6390 - acc: 0.7729 - val_loss: 0.6067 - val_acc: 0.6000
Epoch 6/50
10/10 [==============================] - 100s 10s/step - loss: 0.4944 - acc: 0.7875 - val_loss: 0.4266 - val_acc: 0.8167
Epoch 7/50
10/10 [==============================] - 95s 10s/step - loss: 0.3962 - acc: 0.8091 - val_loss: 0.3276 - val_acc: 0.8500
Epo